In [148]:
using JuMP, Ipopt, Printf, Plots

### Data

In [149]:
xf           = 0.7      ;  # Feed stream propylene mol fraction
xd_target    = 0.95     ; # Target distillate stream Propylene mol fraction
mw_propylene = 42.98 ; # [lb/lbmol]
mw_propane   = 44.1  ; # [lb/lbmol]
mw_feed      = mw_propylene*xf + mw_propane*(1-xf);
mw_dist      = mw_propylene*xd_target + mw_propane*(1-xd_target);

cr           = 1e-6     ; # Reboiler heat cost [$/BTU]
cc           = 3e-05    ; # Condenser cooling cost [$/BTU]
cb           = 0.09     ; # Value of Propylene in bottoms [$/lb]
cbp          = 0.12     ; # Value of Propane in bottoms [$/lb]
cf           = 0.05     ; # Cost per pound of propylene [$/lb]
cfp          = 0.05     ; # Cost per pound of propane [$/lb]
cd           = 0.2      ; # Value of propylene in overhead [$/lb]
cdp          = 0.2      ; # Value of propane in overhead [$/lb]

cf           = cf*mw_propylene; # [$/lbmol]
cfp          = cfp*mw_propane; # [$/lbmol]
cd           = cd*mw_propylene;  # [$/lbmol]
cdp          = cdp*mw_propane; # [$/lbmol]

f            = 1200000  ; # Feed flowrate [lb/day]
f            = f/mw_feed; # [lbmol/day]
alpha        = 1.105    ; # Relative volatility
lambda       = 130      ; # Vaporization latent heat [BTU/lb]
nt           = 94       ; # Total number of trays
rmin         = 1/(alpha-1) * ((xd_target/xf) - alpha*(1-xd_target)/(1-xf)); # Minimum Reflux Ratio

In [150]:
optimum_reflux = Model(solver=IpoptSolver());

### Decision variables

In [151]:
@variable(optimum_reflux, rmin   <= r    <= 20,   start = 1.2*rmin   );
@variable(optimum_reflux, 0      <= nmin <= nt,   start = 0.5*nt     );
@variable(optimum_reflux, 0      <= xb   <= 0.05, start = 0.04       );
@variable(optimum_reflux, 0      <= d    <= f,    start = f*xf       );
@variable(optimum_reflux, 0      <= b    <= f,    start = f*(1-xf)   );
@variable(optimum_reflux, l      >=0,             start = f          );
@variable(optimum_reflux, v      >=0,             start = f          );
@variable(optimum_reflux, mw_bot >=0,             start = mw_propane );

### Minimum number of trays

In [152]:
@NLconstraint(optimum_reflux, c2, (nt-nmin)/(nt+1) == 0.75*(1 - ( (r-rmin)/(r+1) )^(0.5668))); 

### Bottoms mol fraction

In [153]:
@NLconstraint(optimum_reflux, c3, nmin*log(alpha) == log((xd_target/(1-xd_target))*((1-xb)/xb)));
@NLconstraint(optimum_reflux, c1, mw_bot == xb*mw_propylene + (1-xb)*mw_propane);

### Mass balance constraints

In [154]:
@NLconstraint(optimum_reflux, c4, f == d+b);
@NLconstraint(optimum_reflux, c5, f*xf == d*xd_target + b*xb);

### Internal constant flowrates

In [155]:
@NLconstraint(optimum_reflux, c6, l == r*d);
@NLconstraint(optimum_reflux, c7, v == (r+1)*d);

### Objective function

In [156]:
@NLobjective(optimum_reflux, Max, (cd*xd_target*d) + (cbp*(1-xb)*b*mw_bot) 
                                  - (cr*lambda*mw_bot*v + cc*lambda*mw_dist*v) - (cf*xf*f + cfp*(1-xf)*f) ) ;

### Solve problem

In [157]:
status_optimum_reflux = solve(optimum_reflux)

This is Ipopt version 3.12.10, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:       17
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:       19

Total number of variables............................:        8
                     variables with only lower bounds:        3
                variables with lower and upper bounds:        5
                     variables with only upper bounds:        0
Total number of equality constraints.................:        7
Total number of inequality constraints...............:        0
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0 

:Optimal

In [158]:
println("Objective function value = ", getobjectivevalue(optimum_reflux))
@printf " r    : %f\n" getvalue(r)
@printf " nmin : %f\n" getvalue(nmin)
@printf " xb   : %f\n" getvalue(xb)
@printf " d    : %f\n" getvalue(d)
@printf " b    : %f\n" getvalue(b)
@printf " L    : %f\n" getvalue(l)
@printf " V    : %f\n" getvalue(v)
@printf " F    : %f\n" f



Objective function value = 81364.96210365139
 r    : 16.468497
 nmin : 58.979999
 xb   : 0.050000
 d    : 20008.003116
 b    : 7695.385932
 L    : 329501.730790
 V    : 349509.733905
 F    : 27703.389048
